In [37]:
import pandas as pd

import shapely
import shapely.geometry as geometry
import plotly.express as px
import geopandas as gpd
import json

base_path = '../../FINAL data/'
my_data_path = '../../MY data/'

#### final_rosleshoz.csv


In [9]:
file_name = "final_rosleshoz.csv"
path = base_path + "/" + file_name
final_rosselhoz_df = pd.read_csv(path)

In [10]:
final_rosselhoz_df.head()

,new_fire_id,code,coords,locality_distance,lesn,date_start,area_registr,date_end,status,area_all,...,duration,month,status_category,area_union,count_ignitions,points,_typename,geometry,type,ids
0,7463,к-440 (112108),"53° 04' 26"" с.ш.\n102° 32' 53"" в.д.","н.п. Аларь, азимут 38°, удаление 0.5 км",Аларское,2020-04-08,90.0,2020-04-09,ликвидирован 14.04.2020,454,...,1,4,ликвидирован,4.54,1,NaN,Аларь,MULTIPOLYGON (((11416662.69007241 6997742.4409...,НП,[7463]
1,8614,к-1605 (213710),"53° 11' 49"" с.ш.\n102° 14' 53"" в.д.","н.п. Аляты, азимут 140°, удаление 2.4 км",Аларское,2020-06-29,80.0,2020-06-29,ликвидирован 4.07.2020,82,...,0,6,ликвидирован,0.82,1,NaN,Аляты,MULTIPOLYGON (((11382654.2483037 7019963.49327...,НП,[8614]
2,4406,к-600 (169231),"53° 27' 54"" с.ш.\n103° 21' 29"" в.д.","н.п. Ангарский, азимут 32°, удаление 0.6 км",Аларское,2019-04-29,17.0,2019-04-29,ликвидирован 4.05.2019,17,...,0,4,ликвидирован,0.17,1,NaN,Ангарский,MULTIPOLYGON (((11505972.26910861 7069683.9770...,НП,[4406]
3,7567,к-539 (116005),"53° 25' 16"" с.ш.\n102° 57' 43"" в.д.","н.п. Табарсук, азимут 117°, удаление 4.4 км",Аларское,2020-04-09,3.0,2020-04-09,ликвидирован 14.04.2020,28,...,0,4,ликвидирован,0.28,1,NaN,Дута,MULTIPOLYGON (((11461935.80177199 7061539.7212...,НП,[7567]
4,8808,к-1797 (218004),"53° 18' 22"" с.ш.\n103° 00' 29"" в.д.","н.п. Егоровская, азимут 252°, удаление 0.8 км",Аларское,2020-07-06,6.0,2020-07-06,ликвидирован 11.07.2020,7,...,0,7,ликвидирован,0.07,1,NaN,Егоровская,MULTIPOLYGON (((11466936.57961523 7039954.4280...,НП,[8808]


Разница между area и are_union есть


In [11]:
final_rosselhoz_df[final_rosselhoz_df['area_all_km']
                   != final_rosselhoz_df['area_union']][['area_union', 'area_all_km', 'area_all']]

,area_union,area_all_km,area_all
67,0.06,0.03,3
82,0.06,0.03,3
140,487.17,481.15,48115
160,47.20,46.30,4630
201,38.77,38.35,3835
...,...,...,...
9104,0.06,0.03,3
9105,0.06,0.03,3
9106,0.06,0.03,3
9177,0.17,0.14,14


Удалим колонки, дублирующие другие колонки по площади.


In [12]:
final_rosselhoz_df.drop(columns=[
                        "coords", "area_all_km", "area_forest_km", "count_ignitions"], inplace=True)

Ниже видно, что есть id пожара не обязательно равен id нового пожара.


In [13]:
len(final_rosselhoz_df[final_rosselhoz_df['id']
    != final_rosselhoz_df['new_fire_id']])

120

In [14]:
len(final_rosselhoz_df[final_rosselhoz_df['area_all']
    != final_rosselhoz_df['delta_area_all']])

3

Поажры, у которых начало и конец в разных годах. Year — год начала. То же касается месяца.


In [15]:
final_rosselhoz_df[
    (pd.to_datetime(
        final_rosselhoz_df['date_start'], format='%Y-%m-%d').dt.month == 12)
    & (pd.to_datetime(
        final_rosselhoz_df['date_end'], format='%Y-%m-%d').dt.month == 1)][['date_start', 'date_end', 'year']]

,date_start,date_end,year
1296,2018-12-12,2019-01-05,2018
1391,2018-12-24,2019-01-16,2018
1450,2018-12-21,2019-01-09,2018
4539,2018-12-30,2019-01-06,2018
9274,2020-12-04,2021-01-11,2020
9275,2020-12-05,2021-01-11,2020


Удалим дублирование по времени


In [16]:
final_rosselhoz_df.drop(
    columns=["day_start", "day_end", "year", "month", "duration"], inplace=True)

In [17]:
final_rosselhoz_df.head()

,new_fire_id,code,locality_distance,lesn,date_start,area_registr,date_end,status,area_all,area_forest,...,id,geometry_point,buffer,status_category,area_union,points,_typename,geometry,type,ids
0,7463,к-440 (112108),"н.п. Аларь, азимут 38°, удаление 0.5 км",Аларское,2020-04-08,90.0,2020-04-09,ликвидирован 14.04.2020,454,0,...,7463,POINT (11415597.32628477 6996677.077121332),"POLYGON ((11417728.05386004 6998807.804696598,...",ликвидирован,4.54,NaN,Аларь,MULTIPOLYGON (((11416662.69007241 6997742.4409...,НП,[7463]
1,8614,к-1605 (213710),"н.п. Аляты, азимут 140°, удаление 2.4 км",Аларское,2020-06-29,80.0,2020-06-29,ликвидирован 4.07.2020,82,0,...,8614,POINT (11382201.47904679 7019510.724017813),"POLYGON ((11383107.01756061 7020416.262531627,...",ликвидирован,0.82,NaN,Аляты,MULTIPOLYGON (((11382654.2483037 7019963.49327...,НП,[8614]
2,4406,к-600 (169231),"н.п. Ангарский, азимут 32°, удаление 0.6 км",Аларское,2019-04-29,17.0,2019-04-29,ликвидирован 4.05.2019,17,1,...,4406,POINT (11505766.11382733 7069477.82176244),"POLYGON ((11506178.42438989 7069890.132325002,...",ликвидирован,0.17,NaN,Ангарский,MULTIPOLYGON (((11505972.26910861 7069683.9770...,НП,[4406]
3,7567,к-539 (116005),"н.п. Табарсук, азимут 117°, удаление 4.4 км",Аларское,2020-04-09,3.0,2020-04-09,ликвидирован 14.04.2020,28,21,...,7567,POINT (11461671.22664088 7061275.146122432),"POLYGON ((11462200.37690309 7061804.296384646,...",ликвидирован,0.28,NaN,Дута,MULTIPOLYGON (((11461935.80177199 7061539.7212...,НП,[7567]
4,8808,к-1797 (218004),"н.п. Егоровская, азимут 252°, удаление 0.8 км",Аларское,2020-07-06,6.0,2020-07-06,ликвидирован 11.07.2020,7,2,...,8808,POINT (11466804.29204968 7039822.14047591),"POLYGON ((11467068.86718079 7040086.715607016,...",ликвидирован,0.07,NaN,Егоровская,MULTIPOLYGON (((11466936.57961523 7039954.4280...,НП,[8808]


Удалим лишние поля


In [18]:
final_rosselhoz_df.drop(columns=["points", 'ids'], inplace=True)

In [19]:
final_rosselhoz_df.head()

,new_fire_id,code,locality_distance,lesn,date_start,area_registr,date_end,status,area_all,area_forest,...,lon,forest,id,geometry_point,buffer,status_category,area_union,_typename,geometry,type
0,7463,к-440 (112108),"н.п. Аларь, азимут 38°, удаление 0.5 км",Аларское,2020-04-08,90.0,2020-04-09,ликвидирован 14.04.2020,454,0,...,102.548056,0.0,7463,POINT (11415597.32628477 6996677.077121332),"POLYGON ((11417728.05386004 6998807.804696598,...",ликвидирован,4.54,Аларь,MULTIPOLYGON (((11416662.69007241 6997742.4409...,НП
1,8614,к-1605 (213710),"н.п. Аляты, азимут 140°, удаление 2.4 км",Аларское,2020-06-29,80.0,2020-06-29,ликвидирован 4.07.2020,82,0,...,102.248056,0.0,8614,POINT (11382201.47904679 7019510.724017813),"POLYGON ((11383107.01756061 7020416.262531627,...",ликвидирован,0.82,Аляты,MULTIPOLYGON (((11382654.2483037 7019963.49327...,НП
2,4406,к-600 (169231),"н.п. Ангарский, азимут 32°, удаление 0.6 км",Аларское,2019-04-29,17.0,2019-04-29,ликвидирован 4.05.2019,17,1,...,103.358056,1.0,4406,POINT (11505766.11382733 7069477.82176244),"POLYGON ((11506178.42438989 7069890.132325002,...",ликвидирован,0.17,Ангарский,MULTIPOLYGON (((11505972.26910861 7069683.9770...,НП
3,7567,к-539 (116005),"н.п. Табарсук, азимут 117°, удаление 4.4 км",Аларское,2020-04-09,3.0,2020-04-09,ликвидирован 14.04.2020,28,21,...,102.961944,1.0,7567,POINT (11461671.22664088 7061275.146122432),"POLYGON ((11462200.37690309 7061804.296384646,...",ликвидирован,0.28,Дута,MULTIPOLYGON (((11461935.80177199 7061539.7212...,НП
4,8808,к-1797 (218004),"н.п. Егоровская, азимут 252°, удаление 0.8 км",Аларское,2020-07-06,6.0,2020-07-06,ликвидирован 11.07.2020,7,2,...,103.008056,1.0,8808,POINT (11466804.29204968 7039822.14047591),"POLYGON ((11467068.86718079 7040086.715607016,...",ликвидирован,0.07,Егоровская,MULTIPOLYGON (((11466936.57961523 7039954.4280...,НП


Записываем в csv


In [20]:
path = my_data_path + "/" + file_name
final_rosselhoz_df.to_csv(path, index=False)

#### forest_types_processed.csv


#### lesnye_kv_3051.nearest_weather_station-forest_types.csv


In [21]:
file_name = "lesnye_kv_3051.nearest_weather_station-forest_types.csv"
path = base_path + "/Лесные кварталы/" + file_name
nws_df = pd.read_csv(path)

C:\Users\pilno\AppData\Local\Temp\ipykernel_21460\1441391567.py:3: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  nws_df = pd.read_csv(path)


In [22]:
nws_df.head()

,name_in,name_uch_l,in_dacha,name_in_ru,dacha_ru,uch_l_ru,kv_num,lesn1,lesn2,lesn3,...,geom,weather_stations,nearest_weather_station,lesn_path,lesn_path_type,distance,forest_zone,forest_seed_zoning_zone,lesn_path_FH,forest_hazard_classes
0,Alarskoe,Alarskoe,NaN,Аларское,Аларская,Аларское,28,Аларское,Аларское,NaN,...,0106000020E6100000010000000103000000010000000A...,"30616,30606,30703,30617,30611,30603,30605,3061...",30606,аларскоеаларскоеаларская,аларскоеаларскоеаларская,0.0,лесостепная,"сосна-18, ель-10, лиственница-10, кедр-6",alarskoeаларскоеаларская,[4.0]
1,Alarskoe,Alarskoe,NaN,Аларское,Аларская,Аларское,21,Аларское,Аларское,NaN,...,0106000020E61000000100000001030000000100000009...,"30606,30616,30703,30611,30617,30605,30603,3060...",30606,аларскоеаларскоеаларская,аларскоеаларскоеаларская,0.0,лесостепная,"сосна-18, ель-10, лиственница-10, кедр-6",alarskoeаларскоеаларская,[3.0]
2,Alarskoe,Alarskoe,NaN,Аларское,Аларская,Аларское,22,Аларское,Аларское,NaN,...,0106000020E61000000100000001030000000100000008...,"30606,30616,30703,30611,30617,30605,30603,3060...",30606,аларскоеаларскоеаларская,аларскоеаларскоеаларская,0.0,лесостепная,"сосна-18, ель-10, лиственница-10, кедр-6",alarskoeаларскоеаларская,[3.0]
3,Alarskoe,Alarskoe,NaN,Аларское,Аларская,Аларское,23,Аларское,Аларское,NaN,...,0106000020E6100000010000000103000000010000000A...,"30606,30616,30703,30611,30617,30605,30603,3060...",30606,аларскоеаларскоеаларская,аларскоеаларскоеаларская,0.0,лесостепная,"сосна-18, ель-10, лиственница-10, кедр-6",alarskoeаларскоеаларская,[3.0]
4,Alarskoe,Alarskoe,NaN,Аларское,Аларская,Аларское,24,Аларское,Аларское,NaN,...,0106000020E61000000100000001030000000100000009...,"30606,30616,30703,30617,30611,30605,30603,3060...",30606,аларскоеаларскоеаларская,аларскоеаларскоеаларская,0.0,лесостепная,"сосна-18, ель-10, лиственница-10, кедр-6",alarskoeаларскоеаларская,[3.0]


In [23]:
nws_df[nws_df['kv'] != nws_df['kv_num']]

,name_in,name_uch_l,in_dacha,name_in_ru,dacha_ru,uch_l_ru,kv_num,lesn1,lesn2,lesn3,...,geom,weather_stations,nearest_weather_station,lesn_path,lesn_path_type,distance,forest_zone,forest_seed_zoning_zone,lesn_path_FH,forest_hazard_classes


In [24]:
nws_df.drop(columns=["kv_num", 'distance', 'lesn1',
            'lesn2', 'lesn3'], inplace=True)

In [25]:
nws_df.head()

,name_in,name_uch_l,in_dacha,name_in_ru,dacha_ru,uch_l_ru,kv,id,geom,weather_stations,nearest_weather_station,lesn_path,lesn_path_type,forest_zone,forest_seed_zoning_zone,lesn_path_FH,forest_hazard_classes
0,Alarskoe,Alarskoe,NaN,Аларское,Аларская,Аларское,28,22562,0106000020E6100000010000000103000000010000000A...,"30616,30606,30703,30617,30611,30603,30605,3061...",30606,аларскоеаларскоеаларская,аларскоеаларскоеаларская,лесостепная,"сосна-18, ель-10, лиственница-10, кедр-6",alarskoeаларскоеаларская,[4.0]
1,Alarskoe,Alarskoe,NaN,Аларское,Аларская,Аларское,21,22552,0106000020E61000000100000001030000000100000009...,"30606,30616,30703,30611,30617,30605,30603,3060...",30606,аларскоеаларскоеаларская,аларскоеаларскоеаларская,лесостепная,"сосна-18, ель-10, лиственница-10, кедр-6",alarskoeаларскоеаларская,[3.0]
2,Alarskoe,Alarskoe,NaN,Аларское,Аларская,Аларское,22,22553,0106000020E61000000100000001030000000100000008...,"30606,30616,30703,30611,30617,30605,30603,3060...",30606,аларскоеаларскоеаларская,аларскоеаларскоеаларская,лесостепная,"сосна-18, ель-10, лиственница-10, кедр-6",alarskoeаларскоеаларская,[3.0]
3,Alarskoe,Alarskoe,NaN,Аларское,Аларская,Аларское,23,22554,0106000020E6100000010000000103000000010000000A...,"30606,30616,30703,30611,30617,30605,30603,3060...",30606,аларскоеаларскоеаларская,аларскоеаларскоеаларская,лесостепная,"сосна-18, ель-10, лиственница-10, кедр-6",alarskoeаларскоеаларская,[3.0]
4,Alarskoe,Alarskoe,NaN,Аларское,Аларская,Аларское,24,22555,0106000020E61000000100000001030000000100000009...,"30606,30616,30703,30617,30611,30605,30603,3060...",30606,аларскоеаларскоеаларская,аларскоеаларскоеаларская,лесостепная,"сосна-18, ель-10, лиственница-10, кедр-6",alarskoeаларскоеаларская,[3.0]


#### forest_types_processed.csv


In [26]:
file_name = "forest_types_processed.csv"
path = base_path + "/Лесные кварталы/" + file_name
ftp_df = pd.read_csv(path, sep=";")

In [27]:
ftp_df.head()

,name_in,name_in_ru,uch_l_ru,name_in.1,forest_zone,forest_seed_zoning_zone,kv
0,Alarskoe,Аларское,Приморское,Приморская,лесостепная,"сосна-18, ель-10, лиственница-10, кедр-6","1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,1..."
1,Alarskoe,Аларское,Аларское,Аларская,лесостепная,"сосна-18, ель-10, лиственница-10, кедр-6","1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,1..."
2,Alarskoe,Аларское,Алятское,Алятская,лесостепная,"сосна-18, ель-10, лиственница-10, кедр-6","1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,1..."
3,Alarskoe,Аларское,Заречное,Заречная,лесостепная,"сосна-18, ель-10, лиственница-10, кедр-6","1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,1..."
4,Alarskoe,Ангарское,Ушаковское,Худяковская,лесостепная,"сосна-18, ель-10, лиственница-10, кедр-9","1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,1..."


In [28]:
for n in ftp_df['name_in.1']:
    t = ftp_df[ftp_df['name_in.1'] == n]['forest_seed_zoning_zone'].unique()
    print(f'{n} {t}')

# ftp_df['forest_seed_zoning_zone']

Приморская ['сосна-18, ель-10, лиственница-10, кедр-6'
 'сосна-18, ель-10, лиственница-10, кедр-9']
Аларская ['сосна-18, ель-10, лиственница-10, кедр-6']
Алятская ['сосна-18, ель-10, лиственница-10, кедр-6']
Заречная ['сосна-18, ель-10, лиственница-10, кедр-6']
Худяковская ['сосна-18, ель-10, лиственница-10, кедр-9']
Добролетская ['сосна-18, ель-10, лиственница-10, кедр-9']
Ключевская ['сосна-18, ель-10, лиственница-10, кедр-9']
Ангарская ['сосна-18, ель-10, лиственница-10, кедр-9']
Тальцинская ['сосна-18, ель-10, лиственница-10, кедр-9']
Технический участок  №10 ГУСХП ОПХ Иркутское ['сосна-18, ель-10, лиственница-10, кедр-9']
Балаганская ['сосна-18, ель-9, лиственница-10, кедр-9']
Технический участок №1 Ленинский путь ['сосна-18, ель-9, лиственница-10, кедр-9']
Технический участок №3 Балаганский ['сосна-18, ель-9, лиственница-10, кедр-9']
Балаганская ['сосна-18, ель-9, лиственница-10, кедр-9']
Кадинская ['сосна-18, ель-9, лиственница-10, кедр-9']
Карахунская ['сосна-18, ель-9, листвен

In [29]:
ftp_df.rename(columns={'name_in.1': 'dacha'})

,name_in,name_in_ru,uch_l_ru,dacha,forest_zone,forest_seed_zoning_zone,kv
0,Alarskoe,Аларское,Приморское,Приморская,лесостепная,"сосна-18, ель-10, лиственница-10, кедр-6","1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,1..."
1,Alarskoe,Аларское,Аларское,Аларская,лесостепная,"сосна-18, ель-10, лиственница-10, кедр-6","1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,1..."
2,Alarskoe,Аларское,Алятское,Алятская,лесостепная,"сосна-18, ель-10, лиственница-10, кедр-6","1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,1..."
3,Alarskoe,Аларское,Заречное,Заречная,лесостепная,"сосна-18, ель-10, лиственница-10, кедр-6","1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,1..."
4,Alarskoe,Ангарское,Ушаковское,Худяковская,лесостепная,"сосна-18, ель-10, лиственница-10, кедр-9","1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,1..."
...,...,...,...,...,...,...,...
639,Shelehovskoe,Шелеховское,Шелеховское,Олхинская,Южно-Сибирская горная,"сосна-18, ель-12, лиственница-10, кедр-9","65,66,67,68,69,70,71,72,73,74,77,78,79,80,81,8..."
640,Shelehovskoe,Шелеховское,Шелеховское,Подкаменская,Южно-Сибирская горная,"сосна-18, ель-12, лиственница-10, кедр-9","1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,1..."
641,Shelehovskoe,Шелеховское,Шелеховское,Мотская,Южно-Сибирская горная,"сосна-18, ель-12, лиственница-10, кедр-9","1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,1..."
642,Shelehovskoe,Шелеховское,Шелеховское,Олхинская,лесостепная,"сосна-18, ель-10, лиственница-10, кедр-9","1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,1..."


In [30]:
# ftp_df.drop()

In [31]:
ftp_df.head()

,name_in,name_in_ru,uch_l_ru,name_in.1,forest_zone,forest_seed_zoning_zone,kv
0,Alarskoe,Аларское,Приморское,Приморская,лесостепная,"сосна-18, ель-10, лиственница-10, кедр-6","1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,1..."
1,Alarskoe,Аларское,Аларское,Аларская,лесостепная,"сосна-18, ель-10, лиственница-10, кедр-6","1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,1..."
2,Alarskoe,Аларское,Алятское,Алятская,лесостепная,"сосна-18, ель-10, лиственница-10, кедр-6","1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,1..."
3,Alarskoe,Аларское,Заречное,Заречная,лесостепная,"сосна-18, ель-10, лиственница-10, кедр-6","1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,1..."
4,Alarskoe,Ангарское,Ушаковское,Худяковская,лесостепная,"сосна-18, ель-10, лиственница-10, кедр-9","1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,1..."


#### localities_irk_obl.geojson

Geopandas не воспринимает массивы полигонов, необходимо переписать файл

In [85]:
file_name = 'localities_irk_obl.geojson'
path = base_path + "/" + file_name

loc_df = gpd.read_file(path, ignore_geometry=True)

In [86]:
loc_df.head()

,name,type,name_MO,code,distance,ado,id,query,address,geometry,poly,valid,locality
0,Аларь,село сельский,Муниципальное образование «Аларь»,25123902001,44,Аларский,01.���,Аларь село Аларский иркутская область,"Россия, Иркутская область, Аларский район, сел...","[ { ""type"": ""Polygon"", ""coordinates"": [ [ [ 10...",0103000020E6100000010000002E000000726891ED7CA1...,1.0,1
1,Александровск,село сельский,Муниципальное образование «Александровск»,25123904001,10,Аларский,01.���,Александровск село Аларский иркутская область,"Россия, Иркутская область, Аларский район, сел...","[ { ""type"": ""Polygon"", ""coordinates"": [ [ [ 10...",0103000020E61000000100000050000000C30DF8FC30A9...,1.0,1
2,Алзобей,деревня сельский,Муниципальное образование «Аларь»,25123902002,36,Аларский,01.���,Алзобей деревня Аларский иркутская область,"Россия, Иркутская область, Аларский район, дер...","[ { ""type"": ""Polygon"", ""coordinates"": [ [ [ 10...",0103000020E6100000010000001A00000056F0DB10E3A7...,1.0,1
3,Аляты,село сельский,Муниципальное образование «Аляты»,25123907001,55,Аларский,01.���,Аляты село Аларский иркутская область,"Россия, Иркутская область, Аларский район, сел...","[ { ""type"": ""Polygon"", ""coordinates"": [ [ [ 10...",0106000020E61000000200000001030000000100000017...,1.0,1
4,Ангарский,поселок сельский,Муниципальное образование «Ангарский»,25123910001,55,Аларский,01.���,Ангарский поселок Аларский иркутская область,"Россия, Иркутская область, Аларский район, пос...","[ { ""type"": ""Polygon"", ""coordinates"": [ [ [ 10...",0103000020E6100000010000001500000074ECA012D7D5...,1.0,1


In [34]:
def str_to_multipoly(lst):
    # В некоторых строках вместо ',' стоят ';', тогда json не читается
    lst = lst.replace(';', ',')
    j = json.loads(lst)
    polygons = []
    for p in j:
        p = geometry.shape(p)
        polygons.append(p)

    polygons = geometry.MultiPolygon(polygons)

    return geometry.MultiPolygon(polygons)

In [35]:
geom = []

for index, rows in loc_df.iterrows():
    mlp = str_to_multipoly(loc_df.loc[index].geometry)
    # loc_df.at[index, 'geometry'] = mlp
    geom.append(mlp)

In [36]:
gdf = gpd.GeoDataFrame(
    loc_df, geometry=geom, crs="EPSG:4326"
)

In [37]:
gdf.drop(gdf[gdf.geometry.is_empty].index, inplace=True)

In [38]:
gdf.head()

,name,type,name_MO,code,distance,ado,id,query,address,geometry,poly,valid,locality
0,Аларь,село сельский,Муниципальное образование «Аларь»,25123902001,44,Аларский,01.���,Аларь село Аларский иркутская область,"Россия, Иркутская область, Аларский район, сел...","MULTIPOLYGON (((102.52325 53.07457, 102.52335 ...",0103000020E6100000010000002E000000726891ED7CA1...,1.0,1
1,Александровск,село сельский,Муниципальное образование «Александровск»,25123904001,10,Аларский,01.���,Александровск село Аларский иркутская область,"Россия, Иркутская область, Аларский район, сел...","MULTIPOLYGON (((102.64361 53.33077, 102.64394 ...",0103000020E61000000100000050000000C30DF8FC30A9...,1.0,1
2,Алзобей,деревня сельский,Муниципальное образование «Аларь»,25123902002,36,Аларский,01.���,Алзобей деревня Аларский иркутская область,"Россия, Иркутская область, Аларский район, дер...","MULTIPOLYGON (((102.62323 53.11388, 102.62461 ...",0103000020E6100000010000001A00000056F0DB10E3A7...,1.0,1
3,Аляты,село сельский,Муниципальное образование «Аляты»,25123907001,55,Аларский,01.���,Аляты село Аларский иркутская область,"Россия, Иркутская область, Аларский район, сел...","MULTIPOLYGON (((102.22077 53.19755, 102.22384 ...",0106000020E61000000200000001030000000100000017...,1.0,1
4,Ангарский,поселок сельский,Муниципальное образование «Ангарский»,25123910001,55,Аларский,01.���,Ангарский поселок Аларский иркутская область,"Россия, Иркутская область, Аларский район, пос...","MULTIPOLYGON (((103.34125 53.46178, 103.34265 ...",0103000020E6100000010000001500000074ECA012D7D5...,1.0,1


In [39]:
gdf.drop(columns=['query', 'locality'], inplace=True)

In [40]:
gdf[gdf.geometry == None]

,name,type,name_MO,code,distance,ado,id,address,geometry,poly,valid


In [41]:
path = my_data_path + "/" + file_name
gdf.to_file(path, driver='GeoJSON', index=False)

#### user_schema.lesnye_kv_3051.geojson

In [25]:
file_name = "user_schema.lesnye_kv_3051.geojson"

path_to_json = base_path + "Лесные кварталы/" + file_name

gdf = gpd.read_file(path_to_json)

In [26]:
gdf.head()

,name_in,name_uch_l,in_dacha,name_in_ru,dacha_ru,uch_l_ru,kv_num,commentary,lesn1,lesn2,...,id,is_deleted,classname,created_by,edited_by,edited_on,created_on,published,geom:1,geometry
0,Alarskoe,Alarskoe,None,Аларское,Аларская,Аларское,28,None,Аларское,Аларское,...,22562,f,None,None,None,2020-02-21 11:23:01.740,None,f,0106000020E6100000010000000103000000010000000A...,"MULTIPOLYGON (((102.37619 53.25439, 102.37240 ..."
1,Alarskoe,Alarskoe,None,Аларское,Аларская,Аларское,21,None,Аларское,Аларское,...,22552,f,None,None,None,2020-02-21 11:23:01.740,None,f,0106000020E61000000100000001030000000100000009...,"MULTIPOLYGON (((102.22014 53.31237, 102.22171 ..."
2,Alarskoe,Alarskoe,None,Аларское,Аларская,Аларское,22,None,Аларское,Аларское,...,22553,f,None,None,None,2020-02-21 11:23:01.740,None,f,0106000020E61000000100000001030000000100000008...,"MULTIPOLYGON (((102.23036 53.29569, 102.21632 ..."
3,Alarskoe,Alarskoe,None,Аларское,Аларская,Аларское,23,None,Аларское,Аларское,...,22554,f,None,None,None,2020-02-21 11:23:01.740,None,f,0106000020E6100000010000000103000000010000000A...,"MULTIPOLYGON (((102.25694 53.29010, 102.25134 ..."
4,Alarskoe,Alarskoe,None,Аларское,Аларская,Аларское,24,None,Аларское,Аларское,...,22555,f,None,None,None,2020-02-21 11:23:01.740,None,f,0106000020E61000000100000001030000000100000009...,"MULTIPOLYGON (((102.24487 53.28587, 102.24644 ..."


In [27]:
gdf['id'].unique()

array([22562, 22552, 22553, ..., 17483, 17485, 17486], dtype=int64)

In [28]:
gdf.columns

Index(['name_in', 'name_uch_l', 'in_dacha', 'name_in_ru', 'dacha_ru',
       'uch_l_ru', 'kv_num', 'commentary', 'lesn1', 'lesn2', 'lesn3', 'kv',
       'id', 'is_deleted', 'classname', 'created_by', 'edited_by', 'edited_on',
       'created_on', 'published', 'geom:1', 'geometry'],
      dtype='object')

In [29]:
gdf.drop(columns=['commentary', 'lesn1', 'lesn2', 'lesn3',
         'geom:1', 'published', 'created_on', 'edited_on', 'created_by', 'edited_by', 'classname', 'is_deleted', 'kv'],
         inplace=True)

?

In [30]:
gdf.drop(columns={'name_in', 'name_uch_l', 'in_dacha'}, inplace=True)

In [31]:
gdf.drop(gdf[gdf['geometry'].is_empty].index, inplace=True)

In [32]:
gdf.head()

,name_in_ru,dacha_ru,uch_l_ru,kv_num,id,geometry
0,Аларское,Аларская,Аларское,28,22562,"MULTIPOLYGON (((102.37619 53.25439, 102.37240 ..."
1,Аларское,Аларская,Аларское,21,22552,"MULTIPOLYGON (((102.22014 53.31237, 102.22171 ..."
2,Аларское,Аларская,Аларское,22,22553,"MULTIPOLYGON (((102.23036 53.29569, 102.21632 ..."
3,Аларское,Аларская,Аларское,23,22554,"MULTIPOLYGON (((102.25694 53.29010, 102.25134 ..."
4,Аларское,Аларская,Аларское,24,22555,"MULTIPOLYGON (((102.24487 53.28587, 102.24644 ..."


In [33]:
path = my_data_path + "Лесные кварталы/" + file_name
gdf.to_file(path, driver='GeoJSON', index=False)

#### zhd_roads.geojson


In [34]:
file_name = "zhd_roads.geojson"

path_to_json = base_path + "/metadata/" + file_name

df_rail = gpd.read_file(path_to_json)
df_rail.head()


,id,is_deleted,symbol,created_by,edited_by,edited_on,created_on,published,geom:1,geometry
0,1,f,None,57cd206ea47f4dd00900000f,None,2017-04-18 18:20:56.062,2017-04-18 18:20:56.062000+09:00,f,0105000020E61000000100000001020000001300000004...,"LINESTRING (117.01367 52.06945, 117.07109 52.0..."
1,2,f,None,57cd206ea47f4dd00900000f,None,2017-04-18 18:20:56.133,2017-04-18 18:20:56.133000+09:00,f,0105000020E61000000100000001020000002300000058...,"LINESTRING (91.10172 53.57645, 90.95557 53.575..."
2,3,f,None,57cd206ea47f4dd00900000f,None,2017-04-18 18:20:56.135,2017-04-18 18:20:56.135000+09:00,f,0105000020E61000000100000001020000005F000000DA...,"LINESTRING (97.95192 55.94242, 97.84468 55.934..."
3,4,f,None,57cd206ea47f4dd00900000f,None,2017-04-18 18:20:56.136,2017-04-18 18:20:56.136000+09:00,f,0105000020E6100000010000000102000000050000004E...,"LINESTRING (118.12918 50.25033, 118.10700 50.2..."
4,5,f,None,57cd206ea47f4dd00900000f,None,2017-04-18 18:20:56.138,2017-04-18 18:20:56.138000+09:00,f,0105000020E6100000010000000102000000800000003D...,"LINESTRING (114.52750 51.64029, 114.52327 51.6..."


In [35]:
df_rail[df_rail['is_deleted']!='f']

,id,is_deleted,symbol,created_by,edited_by,edited_on,created_on,published,geom:1,geometry


In [36]:
df_rail.drop(columns=['is_deleted', 'published', 'edited_by',
                      'edited_on', 'created_by', 'created_on', 'published', 'symbol'], inplace=True)

In [37]:
df_rail.head()

,id,geom:1,geometry
0,1,0105000020E61000000100000001020000001300000004...,"LINESTRING (117.01367 52.06945, 117.07109 52.0..."
1,2,0105000020E61000000100000001020000002300000058...,"LINESTRING (91.10172 53.57645, 90.95557 53.575..."
2,3,0105000020E61000000100000001020000005F000000DA...,"LINESTRING (97.95192 55.94242, 97.84468 55.934..."
3,4,0105000020E6100000010000000102000000050000004E...,"LINESTRING (118.12918 50.25033, 118.10700 50.2..."
4,5,0105000020E6100000010000000102000000800000003D...,"LINESTRING (114.52750 51.64029, 114.52327 51.6..."


In [38]:
df_rail[df_rail['geometry'].is_empty]

,id,geom:1,geometry


In [39]:
path = my_data_path + "metadata/" + file_name
df_rail.to_file(path, driver='GeoJSON', index=False)

#### rivers.geojson

In [40]:
file_name = 'rivers.geojson'

path_to_json = base_path + "/metadata/" + file_name
# ignore_geometry=True
df_rivers = gpd.read_file(path_to_json)

In [41]:
df_rivers.head()

,name,basin,basin2,linewidth,sq,id,is_deleted,created_by,edited_by,edited_on,created_on,published,line,geometry
0,Артюгина,Енисей,Артюгина,0.3,A01,1,f,50f7a1d80d58140037000006,None,2016-05-27 16:52:38.553,2016-05-27 16:52:38.553000+08:00,f,0105000020E61000000100000001020000002900000080...,"LINESTRING (87.28161 63.35722, 87.25699 63.329..."
1,Верх. Сарчиха,Енисей,Верх. Сарчиха,0.3,A01,2,f,50f7a1d80d58140037000006,None,2016-05-27 16:52:40.242,2016-05-27 16:52:40.242000+08:00,f,0105000020E61000000100000001020000005A00000097...,"LINESTRING (88.93790 62.45754, 88.90587 62.452..."
2,Каменный Дубчес,Енисей,Дубчес,0.3,A01,3,f,50f7a1d80d58140037000006,None,2016-05-27 16:52:40.272,2016-05-27 16:52:40.272000+08:00,f,0105000020E6100000010000000102000000C100000053...,"LINESTRING (88.36790 61.39750, 88.34564 61.411..."
3,Елогуй,Енисей,Елогуй,0.3,A01,8,f,50f7a1d80d58140037000006,None,2016-05-27 16:52:40.284,2016-05-27 16:52:40.284000+08:00,f,0105000020E61000000100000001020000005C0000008C...,"LINESTRING (85.40091 61.99233, 85.36168 62.007..."
4,Делтула,Енисей,Бахта,0.3,A02,19,f,50f7a1d80d58140037000006,None,2016-05-27 16:52:40.305,2016-05-27 16:52:40.305000+08:00,f,0105000020E61000000100000001020000000A000000D5...,"LINESTRING (90.38151 63.67763, 90.42314 63.727..."


In [42]:
len(df_rivers)

675

In [43]:
df_rivers.drop(columns=['is_deleted', 'published', 'edited_by',
                        'edited_on', 'created_by', 'created_on', 'published'], inplace=True)

In [44]:
df_rivers.head()

,name,basin,basin2,linewidth,sq,id,line,geometry
0,Артюгина,Енисей,Артюгина,0.3,A01,1,0105000020E61000000100000001020000002900000080...,"LINESTRING (87.28161 63.35722, 87.25699 63.329..."
1,Верх. Сарчиха,Енисей,Верх. Сарчиха,0.3,A01,2,0105000020E61000000100000001020000005A00000097...,"LINESTRING (88.93790 62.45754, 88.90587 62.452..."
2,Каменный Дубчес,Енисей,Дубчес,0.3,A01,3,0105000020E6100000010000000102000000C100000053...,"LINESTRING (88.36790 61.39750, 88.34564 61.411..."
3,Елогуй,Енисей,Елогуй,0.3,A01,8,0105000020E61000000100000001020000005C0000008C...,"LINESTRING (85.40091 61.99233, 85.36168 62.007..."
4,Делтула,Енисей,Бахта,0.3,A02,19,0105000020E61000000100000001020000000A000000D5...,"LINESTRING (90.38151 63.67763, 90.42314 63.727..."


In [45]:
df_rivers['linewidth'].unique()

array([0.3, 0.4, 1.4, 0.7, 0.5, 0.9, 1. , 0.8, 0.6, 1.1, 1.2])

In [46]:
df_rivers['sq'].unique()

array(['A01', 'A02', 'A07', 'A08', 'A06', 'A04', 'A05', 'A03', 'F12',
       'B05', 'C03', 'A10', 'A09', 'B03', 'A11', 'C11', 'B01', 'B04',
       'B02', 'F05', 'B07', 'B06', 'C12', 'B09', 'B08', 'B10', 'C06',
       'B11', 'D06', 'E11', 'C01', 'C02', 'F06', 'C09', 'C04', 'C05',
       'E07', 'G05', 'C07', 'C10', 'C08', 'D01', 'D07', 'D03', 'D05',
       'D02', 'E03', 'D04', 'D10', 'D08', 'D11', 'D09', 'G06', 'D12',
       'F08', 'E01', 'E02', 'E06', 'E04', 'F11', 'E05', 'F03', 'E08',
       'E09', 'E10', 'G01', 'F01', 'E12', 'F02', 'F04', 'F07', 'F09',
       'F10', 'G03', 'G04', 'G02', 'G08', 'G07', 'G09', 'G10', 'G11',
       'H01', 'G12', 'H02', 'H03', 'H05', 'H04', 'H06', 'I01', 'H07',
       'H09', 'H08', 'H10', 'H11', 'I02', 'I03', 'I05', 'I04', 'I06',
       'J01', 'I07', 'I08', 'I10', 'I11', 'J02', 'J05', 'J04', 'J03',
       'J07', 'J08', 'J10', 'J11', 'K04', 'K05', 'M04'], dtype=object)

In [47]:
path = my_data_path + "metadata/" + file_name
df_rivers.to_file(path, driver='GeoJSON', index=False)

#### auto_roads.geojson

In [48]:
file_name = 'auto_roads.geojson'

path_to_json = base_path + "/metadata/" + file_name
# ignore_geometry=True
df_roads = gpd.read_file(path_to_json)

In [49]:
df_roads.head()

,type,id,is_deleted,symbol,created_by,edited_by,edited_on,created_on,published,geom:1,geometry
0,федеральная,122,f,None,57cd206ea47f4dd00900000f,None,2017-04-18 18:15:04.311,2017-04-18 18:15:04.311000+09:00,f,0105000020E61000000100000001020000006E00000025...,"LINESTRING (103.53249 48.81194, 103.50655 48.8..."
1,прочая,123,f,None,57cd206ea47f4dd00900000f,None,2017-04-18 18:15:04.574,2017-04-18 18:15:04.574000+09:00,f,0105000020E61000000100000001020000000800000058...,"LINESTRING (98.64292 49.51586, 98.62168 49.495..."
2,прочая,124,f,None,57cd206ea47f4dd00900000f,None,2017-04-18 18:15:04.576,2017-04-18 18:15:04.576000+09:00,f,0105000020E61000000100000001020000004500000001...,"LINESTRING (96.32309 46.41569, 96.34554 46.462..."
3,федеральная,125,f,None,57cd206ea47f4dd00900000f,None,2017-04-18 18:15:04.577,2017-04-18 18:15:04.577000+09:00,f,0105000020E61000000100000001020000002800000024...,"LINESTRING (102.37421 47.39656, 102.35455 47.3..."
4,федеральная,126,f,None,57cd206ea47f4dd00900000f,None,2017-04-18 18:15:04.579,2017-04-18 18:15:04.579000+09:00,f,0105000020E61000000100000001020000000C00000097...,"LINESTRING (104.99265 44.00482, 104.96229 43.9..."


In [50]:
df_roads.drop(columns={'edited_by', 'created_by', 'edited_on',
              'created_on', 'published', 'is_deleted', 'symbol', 'geom:1'}, inplace=True)
df_roads.head()

,type,id,geometry
0,федеральная,122,"LINESTRING (103.53249 48.81194, 103.50655 48.8..."
1,прочая,123,"LINESTRING (98.64292 49.51586, 98.62168 49.495..."
2,прочая,124,"LINESTRING (96.32309 46.41569, 96.34554 46.462..."
3,федеральная,125,"LINESTRING (102.37421 47.39656, 102.35455 47.3..."
4,федеральная,126,"LINESTRING (104.99265 44.00482, 104.96229 43.9..."


In [ ]:
path = my_data_path + "metadata/" + file_name
df_roads.to_file(path, driver='GeoJSON', index=False)

#### distance from urban area

In [109]:
file_name = 'distance from urban area.geojson'

path_to_json = base_path + "/metadata/" + file_name
# ignore_geometry=True
df_urban_dist = gpd.read_file(path_to_json, ignore_geometry=True)

In [110]:
df_urban_dist.head()

,name,type,name_MO,code,distance,ado,id,query,address,geometry,poly,valid,locality,layer,path
0,Аларь,село сельский,Муниципальное образование «Аларь»,25123902001,44,Аларский,01.���,Аларь село Аларский иркутская область,"Россия, Иркутская область, Аларский район, сел...","(1:,...)",0103000020E6100000010000002E000000726891ED7CA1...,1.0,1,5000,Polygon?crs=EPSG:32648&field=name:string(255)&...
1,Аларь,село сельский,Муниципальное образование «Аларь»,25123902001,44,Аларский,01.���,Аларь село Аларский иркутская область,"Россия, Иркутская область, Аларский район, сел...","(1:,...)",0103000020E6100000010000002E000000726891ED7CA1...,1.0,1,50000,Polygon?crs=EPSG:32648&field=name:string(255)&...
2,Аларь,село сельский,Муниципальное образование «Аларь»,25123902001,44,Аларский,01.���,Аларь село Аларский иркутская область,"Россия, Иркутская область, Аларский район, сел...","(1:,...)",0103000020E6100000010000002E000000726891ED7CA1...,1.0,1,30000,Polygon?crs=EPSG:32648&field=name:string(255)&...
3,Аларь,село сельский,Муниципальное образование «Аларь»,25123902001,44,Аларский,01.���,Аларь село Аларский иркутская область,"Россия, Иркутская область, Аларский район, сел...","(1:,...)",0103000020E6100000010000002E000000726891ED7CA1...,1.0,1,25000,Polygon?crs=EPSG:32648&field=name:string(255)&...
4,Аларь,село сельский,Муниципальное образование «Аларь»,25123902001,44,Аларский,01.���,Аларь село Аларский иркутская область,"Россия, Иркутская область, Аларский район, сел...","(1:,...)",0103000020E6100000010000002E000000726891ED7CA1...,1.0,1,2500,Polygon?crs=EPSG:32648&field=name:string(255)&...


In [111]:
df_urban_dist['poly'].unique()

array(['0103000020E6100000010000002E000000726891ED7CA15940711DE38A8B894A402497FF907EA15940EB6E9EEA90894A40F71C588E90A15940DA71C3EFA6894A402828452BF7A15940A5F5B704E0894A40AAEFFCA204A25940F25EB532E1894A40832D76FBACA2594068E55E60568A4A40BAF3C473B6A2594082531F48DE894A40CD203EB0E3A25940F0A5F0A0D9894A4031CB9E0436A3594009E066F162894A40E5400FB56DA35940DAC4C9FD0E894A404276DEC666A35940AF78EA9106894A4039B709F7CAA359407CCEDDAE97884A40CB9BC3B5DAA35940B16D516683884A40F8156BB8C8A359401CCD91955F884A4033DAAA24B2A359400DC4B29943884A4015DEE522BEA35940D5C9198A3B884A405ED89AADBCA359405D143DF031884A40A33D5E4887A3594019A721AAF0874A40A0FCDD3B6AA359401763601DC7874A408252B4722FA35940530438BD8B874A40D0419770E8A25940D4282499D5874A4045B6F3FDD4A259401EF64201DB874A40E542E55FCBA259403E9468C9E3874A404DF2237EC5A25940533D997FF4874A40C3B7B06EBCA2594012656F29E7874A407C9A931799A259407FBDC282FB874A40230F441669A259408AABCABE2B884A40D384ED2763A25940486B0C3A21884A40E2E5E95C51A2594014CC988235884A400133DFC14FA25940F12900C633884A

In [112]:
df_urban_dist.drop(columns={'query', 'address', 'valid', 'locality', 'path'}, inplace=True)

In [113]:
df_urban_dist.head()

,name,type,name_MO,code,distance,ado,id,geometry,poly,layer
0,Аларь,село сельский,Муниципальное образование «Аларь»,25123902001,44,Аларский,01.���,"(1:,...)",0103000020E6100000010000002E000000726891ED7CA1...,5000
1,Аларь,село сельский,Муниципальное образование «Аларь»,25123902001,44,Аларский,01.���,"(1:,...)",0103000020E6100000010000002E000000726891ED7CA1...,50000
2,Аларь,село сельский,Муниципальное образование «Аларь»,25123902001,44,Аларский,01.���,"(1:,...)",0103000020E6100000010000002E000000726891ED7CA1...,30000
3,Аларь,село сельский,Муниципальное образование «Аларь»,25123902001,44,Аларский,01.���,"(1:,...)",0103000020E6100000010000002E000000726891ED7CA1...,25000
4,Аларь,село сельский,Муниципальное образование «Аларь»,25123902001,44,Аларский,01.���,"(1:,...)",0103000020E6100000010000002E000000726891ED7CA1...,2500


In [114]:
p = df_urban_dist['poly'][0]
pg = gpd.GeoSeries.from_wkb(df_urban_dist['poly'])

In [115]:
pg

0     POLYGON ((102.52325 53.07457, 102.52335 53.074...
1     POLYGON ((102.52325 53.07457, 102.52335 53.074...
2     POLYGON ((102.52325 53.07457, 102.52335 53.074...
3     POLYGON ((102.52325 53.07457, 102.52335 53.074...
4     POLYGON ((102.52325 53.07457, 102.52335 53.074...
5     POLYGON ((102.52325 53.07457, 102.52335 53.074...
6     POLYGON ((102.52325 53.07457, 102.52335 53.074...
7     POLYGON ((102.52325 53.07457, 102.52335 53.074...
8     POLYGON ((102.52325 53.07457, 102.52335 53.074...
9     POLYGON ((102.52325 53.07457, 102.52335 53.074...
10    POLYGON ((102.52325 53.07457, 102.52335 53.074...
dtype: geometry

In [116]:
df_urban_dist.geometry = pg

In [117]:
df_urban_dist

,name,type,name_MO,code,distance,ado,id,geometry,poly,layer
0,Аларь,село сельский,Муниципальное образование «Аларь»,25123902001,44,Аларский,01.���,"POLYGON ((102.52325 53.07457, 102.52335 53.074...",0103000020E6100000010000002E000000726891ED7CA1...,5000
1,Аларь,село сельский,Муниципальное образование «Аларь»,25123902001,44,Аларский,01.���,"POLYGON ((102.52325 53.07457, 102.52335 53.074...",0103000020E6100000010000002E000000726891ED7CA1...,50000
2,Аларь,село сельский,Муниципальное образование «Аларь»,25123902001,44,Аларский,01.���,"POLYGON ((102.52325 53.07457, 102.52335 53.074...",0103000020E6100000010000002E000000726891ED7CA1...,30000
3,Аларь,село сельский,Муниципальное образование «Аларь»,25123902001,44,Аларский,01.���,"POLYGON ((102.52325 53.07457, 102.52335 53.074...",0103000020E6100000010000002E000000726891ED7CA1...,25000
4,Аларь,село сельский,Муниципальное образование «Аларь»,25123902001,44,Аларский,01.���,"POLYGON ((102.52325 53.07457, 102.52335 53.074...",0103000020E6100000010000002E000000726891ED7CA1...,2500
5,Аларь,село сельский,Муниципальное образование «Аларь»,25123902001,44,Аларский,01.���,"POLYGON ((102.52325 53.07457, 102.52335 53.074...",0103000020E6100000010000002E000000726891ED7CA1...,7500
6,Аларь,село сельский,Муниципальное образование «Аларь»,25123902001,44,Аларский,01.���,"POLYGON ((102.52325 53.07457, 102.52335 53.074...",0103000020E6100000010000002E000000726891ED7CA1...,20000
7,Аларь,село сельский,Муниципальное образование «Аларь»,25123902001,44,Аларский,01.���,"POLYGON ((102.52325 53.07457, 102.52335 53.074...",0103000020E6100000010000002E000000726891ED7CA1...,15000
8,Аларь,село сельский,Муниципальное образование «Аларь»,25123902001,44,Аларский,01.���,"POLYGON ((102.52325 53.07457, 102.52335 53.074...",0103000020E6100000010000002E000000726891ED7CA1...,10000
9,Аларь,село сельский,Муниципальное образование «Аларь»,25123902001,44,Аларский,01.���,"POLYGON ((102.52325 53.07457, 102.52335 53.074...",0103000020E6100000010000002E000000726891ED7CA1...,180000


In [118]:
df_urban_dist.drop(columns=['poly', 'layer', 'ado'], inplace=True)

In [119]:
df_urban_dist.head()

,name,type,name_MO,code,distance,id,geometry
0,Аларь,село сельский,Муниципальное образование «Аларь»,25123902001,44,01.���,"POLYGON ((102.52325 53.07457, 102.52335 53.074..."
1,Аларь,село сельский,Муниципальное образование «Аларь»,25123902001,44,01.���,"POLYGON ((102.52325 53.07457, 102.52335 53.074..."
2,Аларь,село сельский,Муниципальное образование «Аларь»,25123902001,44,01.���,"POLYGON ((102.52325 53.07457, 102.52335 53.074..."
3,Аларь,село сельский,Муниципальное образование «Аларь»,25123902001,44,01.���,"POLYGON ((102.52325 53.07457, 102.52335 53.074..."
4,Аларь,село сельский,Муниципальное образование «Аларь»,25123902001,44,01.���,"POLYGON ((102.52325 53.07457, 102.52335 53.074..."


In [120]:
df_urban_dist = gpd.GeoDataFrame(
    df_urban_dist,
     # geometry=geom,
      crs="EPSG:4326"
)

In [121]:
path = my_data_path + "metadata/" + file_name
df_urban_dist.to_file(path, driver='GeoJSON', index=False)

#### loclities buffers

In [85]:
fileName = 'localities_merged.geojson'

path_to_json = my_data_path + "/MY buffers/" + fileName
# ignore_geometry=True
df_loc_buf = gpd.read_file(path_to_json)

df_loc_buf.head()

,name,type,name_MO,code,distance,ado,id,address,poly,valid,layer,path,geometry
0,Аларь,село сельский,Муниципальное образование «Аларь»,25123902001,44,Аларский,01.���,"Россия, Иркутская область, Аларский район, сел...",0103000020E6100000010000002E000000726891ED7CA1...,1.0,10km,MultiPolygon?crs=EPSG:3857&field=name:string(0...,"MULTIPOLYGON (((100.44180 53.59649, 100.44197 ..."
1,Аларь,село сельский,Муниципальное образование «Аларь»,25123902001,44,Аларский,01.���,"Россия, Иркутская область, Аларский район, сел...",0103000020E6100000010000002E000000726891ED7CA1...,1.0,10km,MultiPolygon?crs=EPSG:3857&field=name:string(0...,"MULTIPOLYGON (((98.11915 53.64630, 98.11917 53..."
2,Аларь,село сельский,Муниципальное образование «Аларь»,25123902001,44,Аларский,01.���,"Россия, Иркутская область, Аларский район, сел...",0103000020E6100000010000002E000000726891ED7CA1...,1.0,10km,MultiPolygon?crs=EPSG:3857&field=name:string(0...,"MULTIPOLYGON (((98.16586 53.87099, 98.16606 53..."
3,Аларь,село сельский,Муниципальное образование «Аларь»,25123902001,44,Аларский,01.���,"Россия, Иркутская область, Аларский район, сел...",0103000020E6100000010000002E000000726891ED7CA1...,1.0,10km,MultiPolygon?crs=EPSG:3857&field=name:string(0...,"MULTIPOLYGON (((99.82177 53.95239, 99.82274 53..."
4,Аларь,село сельский,Муниципальное образование «Аларь»,25123902001,44,Аларский,01.���,"Россия, Иркутская область, Аларский район, сел...",0103000020E6100000010000002E000000726891ED7CA1...,1.0,10km,MultiPolygon?crs=EPSG:3857&field=name:string(0...,"MULTIPOLYGON (((100.48422 54.08847, 100.48423 ..."


In [86]:
df_loc_buf.drop(columns=['path', 'valid', 'ado', 'distance', 'code', 'name_MO', 'type', 'address', 'name', 'id'], inplace=True)

In [94]:
wkb = df_loc_buf.geometry.to_wkb()

In [95]:
df_loc_buf.drop(columns=['geometry'], inplace=True)

In [96]:
df_loc_buf.head()

,poly,layer
0,0103000020E6100000010000002E000000726891ED7CA1...,10km
1,0103000020E6100000010000002E000000726891ED7CA1...,10km
2,0103000020E6100000010000002E000000726891ED7CA1...,10km
3,0103000020E6100000010000002E000000726891ED7CA1...,10km
4,0103000020E6100000010000002E000000726891ED7CA1...,10km


In [100]:
loc_buf_pd = pd.DataFrame(df_loc_buf)
loc_buf_pd.head()

,poly,layer
0,0103000020E6100000010000002E000000726891ED7CA1...,10km
1,0103000020E6100000010000002E000000726891ED7CA1...,10km
2,0103000020E6100000010000002E000000726891ED7CA1...,10km
3,0103000020E6100000010000002E000000726891ED7CA1...,10km
4,0103000020E6100000010000002E000000726891ED7CA1...,10km


In [101]:
file_name1 = 'no_geometry.geojson'
file_name2 = 'no_poly.geojson'
# path = my_data_path + "/MY buffers/" + file_name2
# df_loc_buf.to_file(path, driver='GeoJSON',index=False)

path = my_data_path + "/MY buffers/" + file_name1

In [102]:
# df_loc_buf.to_file(path, )
loc_buf_pd.to_json(path, index=False)

In [26]:
str(loc_buf_pd.layer[0])

'10km'

In [106]:
# df_loc_buf = gpd.read_file(path)
df_loc_buf = pd.read_json(path)

df_loc_buf.head()

,poly,layer
0,0103000020E6100000010000002E000000726891ED7CA1...,10km
1,0103000020E6100000010000002E000000726891ED7CA1...,10km
2,0103000020E6100000010000002E000000726891ED7CA1...,10km
3,0103000020E6100000010000002E000000726891ED7CA1...,10km
4,0103000020E6100000010000002E000000726891ED7CA1...,10km


In [110]:
gdf = gpd.GeoDataFrame(df_loc_buf)
gdf.head()

,poly,layer
0,0103000020E6100000010000002E000000726891ED7CA1...,10km
1,0103000020E6100000010000002E000000726891ED7CA1...,10km
2,0103000020E6100000010000002E000000726891ED7CA1...,10km
3,0103000020E6100000010000002E000000726891ED7CA1...,10km
4,0103000020E6100000010000002E000000726891ED7CA1...,10km


In [118]:
# gdf._set_geometry(wkb)
geoms = []
for shape in gdf.poly:
    geoms.append(shapely.from_wkb(shape))

geoms
gdf.set_geometry(geoms, inplace=True)
gdf.head()

,poly,layer,geometry
0,0103000020E6100000010000002E000000726891ED7CA1...,10km,"POLYGON ((102.52325 53.07457, 102.52335 53.074..."
1,0103000020E6100000010000002E000000726891ED7CA1...,10km,"POLYGON ((102.52325 53.07457, 102.52335 53.074..."
2,0103000020E6100000010000002E000000726891ED7CA1...,10km,"POLYGON ((102.52325 53.07457, 102.52335 53.074..."
3,0103000020E6100000010000002E000000726891ED7CA1...,10km,"POLYGON ((102.52325 53.07457, 102.52335 53.074..."
4,0103000020E6100000010000002E000000726891ED7CA1...,10km,"POLYGON ((102.52325 53.07457, 102.52335 53.074..."
